## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import sys

sys.path.append("../")

#config = os.path.join("..","config.py")
#from .. import config


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
city_path = os.path.join("..", "Weather_Database", "WeatherPy_database.csv")
city_data_df = pd.read_csv(city_path)
#city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,73.54,89,100,2.35,overcast clouds
1,1,Dongsheng,CN,39.8161,109.9776,69.01,34,100,15.46,overcast clouds
2,2,Berlevag,NO,70.8578,29.0864,35.08,72,100,26.02,light snow
3,3,Dikson,RU,73.5069,80.5464,28.83,85,99,4.34,overcast clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,41.11,87,20,1.59,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#from VacationPy line 11
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Marzuq,YE,14.4000,46.4667,76.69,39,23,3.40,few clouds
15,15,Kendari,ID,-3.9450,122.4989,79.43,86,100,3.71,overcast clouds
19,19,Pochutla,MX,15.7432,-96.4661,85.82,69,3,2.89,clear sky
23,23,Pierre,US,44.3683,-100.3510,75.99,76,90,8.99,overcast clouds
28,28,Samana,DO,19.2056,-69.3369,75.56,89,100,6.53,overcast clouds
29,29,Lazaro Cardenas,MX,17.9583,-102.2000,78.35,58,13,4.65,few clouds
33,33,Makakilo City,US,21.3469,-158.0858,82.72,51,75,10.36,broken clouds
34,34,Mareeba,AU,-17.0000,145.4333,76.44,58,66,7.63,broken clouds
36,36,Cururupu,BR,-1.8283,-44.8683,75.65,94,16,3.27,few clouds
39,39,Butaritari,KI,3.0707,172.7902,81.82,77,100,22.12,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
#preferred_cities_df.to_csv('prefcities.csv')

City_ID                227
City                   227
Country                227
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()


City_ID                227
City                   227
Country                227
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Marzuq,YE,76.69,few clouds,14.4000,46.4667,
15,Kendari,ID,79.43,overcast clouds,-3.9450,122.4989,
19,Pochutla,MX,85.82,clear sky,15.7432,-96.4661,
23,Pierre,US,75.99,overcast clouds,44.3683,-100.3510,
28,Samana,DO,75.56,overcast clouds,19.2056,-69.3369,
29,Lazaro Cardenas,MX,78.35,few clouds,17.9583,-102.2000,
33,Makakilo City,US,82.72,broken clouds,21.3469,-158.0858,
34,Mareeba,AU,76.44,broken clouds,-17.0000,145.4333,
36,Cururupu,BR,75.65,few clouds,-1.8283,-44.8683,
39,Butaritari,KI,81.82,overcast clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df.loc[:,"Hotel Name"] != "")]


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
15,-3.9450,122.4989
19,15.7432,-96.4661
23,44.3683,-100.3510
28,19.2056,-69.3369
29,17.9583,-102.2000
...,...,...
688,48.0000,122.7167
691,24.0934,32.9070
694,44.6995,-73.4529
695,-2.9701,119.8978


In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))